In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Check Fold


> ## 📌 ใช้ค่า MCV, MCH, Hb เทรน ML 
> ## 📌 ใช้ภาพ 2dFFT image 3 channels 

- 🩸แบบ 2 โรค (E, M)


### 📂 DATA : ค่าจากผล Lab

In [2]:
# Read excel file with sheet name
dict_df = pd.read_excel('/home/kannika/code/Rheology2023/Rheology_Blood/All lab results (Blood viscosity work)3.xlsx', 
                   sheet_name=['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม', 'Normal'], header=2)
dict_df

{'Patients_ตัดม้าม':         No.  Code         HN        AGE          PC  PS activity    Free PS  \
 0         1  HN01  2045512.0  29.000000   76.800000    43.000000  58.600000   
 1         2  HN02  4045280.0  21.000000  100.400000    58.800000  70.600000   
 2         3  HN03  3024257.0  41.000000   75.600000    52.500000  57.900000   
 3         4  HN04  1257101.0  40.000000   31.700000    57.700000  59.300000   
 4         6  HN06  1797136.0  32.000000   43.700000    41.100000  77.300000   
 5         7  HN07  2234214.0  50.000000   66.400000    54.500000  62.100000   
 6        12  HN12  1110168.0  45.000000   87.800000    39.300000  60.000000   
 7        13  HN13  3949905.0  23.000000   50.400000    37.000000  83.600000   
 8        16  HN16  1665191.0  35.000000   79.000000    47.400000  59.100000   
 9        17  HN17  2272494.0  27.000000   47.600000    34.200000  59.600000   
 10       19  HN19  1513782.0  36.000000   55.500000    83.000000  76.600000   
 11       23  HN23  

In [6]:
''''
subclass
Patients_ตัดม้าม: Splenectomy
Patients_ไม่ตัดม้าม: No_Splenectomy
Normal
'''

dict_sheet =  ['Patients_ตัดม้าม','Patients_ไม่ตัดม้าม']
lstdict_code  = []
for sheet in dict_sheet:
    df_sheet = dict_df[sheet]
    df_sheet["sheet"] = sheet
    dict_code = df_sheet[['Code','MCV', 'MCH', 'Hb', 'typeBEvsBM', 'sheet']]
    #dict_code = dict_code[dict_code['Code'].notna()]
    _dict_code = dict_code[(dict_code['Code'].notna())&(dict_code['MCV'].notna())&(dict_code['MCH'].notna())
                         &dict_code['Hb'].notna()]
    __dict_code = _dict_code[(_dict_code['MCV']!= '-')|(_dict_code['MCH']!= '-')
                         |(_dict_code['Hb']!= '-')]
    lstdict_code.append(__dict_code)
    print(dict_df[sheet].shape)

print(len(lstdict_code))
df_dictCode = pd.concat(lstdict_code, ignore_index=True)
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ตัดม้าม','Splenectomy')
df_dictCode["sheet"] = df_dictCode['sheet'].str.replace('Patients_ไม่ตัดม้าม','No_Splenectomy')
print(df_dictCode.shape)
df_dictCode.head()

(27, 52)
(27, 52)
2
(44, 6)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet
0,HN01,75.8,23.1,6.0,E,Splenectomy
1,HN02,73.5,22.4,7.2,E,Splenectomy
2,HN03,77.3,24.3,9.0,E,Splenectomy
3,HN04,79.8,25.6,7.6,E,Splenectomy
4,HN06,78.5,25.9,8.2,E,Splenectomy


In [8]:
print(len(list(set(df_dictCode["Code"]))))
print(set(df_dictCode["sheet"]))
print(set(df_dictCode["typeBEvsBM"]))

44
{'Splenectomy', 'No_Splenectomy'}
{'M', 'E', 'AEBart'}


In [9]:
df_dictCode.isnull().values.any()

False

In [10]:
df_dictCode.to_csv("/home/kannika/code/Rheology2023/Rheology_Blood/All_lab_typeBEvsBM.csv")

In [46]:
EM_list = list(set(df_dictCode["typeBEvsBM"]))
EM_list.sort()
for j in range(len(EM_list)):
    classes_j = EM_list[j]
    df_j = df_dictCode[df_dictCode["typeBEvsBM"]==classes_j]
    print(f"[INFO]: Classes: {classes_j} ==> {df_j.shape[0]}")

[INFO]: Classes: AEBart ==> 2
[INFO]: Classes: E ==> 31
[INFO]: Classes: M ==> 11


In [102]:
31+11

42

----------------------------------------------------------------------------

#### 🩸2d FFT Images

In [23]:
# #df_EM = df_dictCode[df_dictCode["typeBEvsBM"]!='AEBart'].reset_index(drop=True)
# print(df_EM.shape)
# df_EM.head()

In [24]:
## 1. Create Dict
lst_Code = df_dictCode["Code"].tolist()
lst_EM = df_dictCode["typeBEvsBM"].tolist()
dict_code = zip(lst_Code, lst_EM)
output_dict = dict(dict_code)
print(output_dict)

{'HN01': 'E', 'HN02': 'E', 'HN03': 'E', 'HN04': 'E', 'HN06': 'E', 'HN07': 'E', 'HN12': 'E', 'HN13': 'E', 'HN16': 'E', 'HN17': 'E', 'HN19': 'M', 'HN23': 'E', 'HN24': 'M', 'HN25': 'M', 'HN27': 'E', 'HN28': 'M', 'HN32': 'E', 'HN36': 'M', 'HN37': 'AEBart', 'HN39': 'E', 'HN40': 'M', 'HN44': 'E', 'HN05': 'E', 'HN08': 'E', 'HN09': 'E', 'HN10': 'M', 'HN11': 'M', 'HN14': 'E', 'HN15': 'AEBart', 'HN18': 'E', 'HN20': 'E', 'HN21': 'E', 'HN22': 'M', 'HN26': 'E', 'HN29': 'E', 'HN30': 'E', 'HN31': 'E', 'HN33': 'E', 'HN34': 'E', 'HN35': 'E', 'HN38': 'E', 'HN41': 'M', 'HN42': 'E', 'HN43': 'M'}


> #### 6-fold

In [25]:
DF_A_res = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_2dFFTdataset_3channels_6Fold_split2class-ver2.csv')
print(DF_A_res.shape)
DF_2dfft6fold = DF_A_res[DF_A_res['subclass']!='Normal']
print("Classes: typeBEvsBM ==> ", DF_2dfft6fold.shape)
DF_2dfft6fold.head()

(268, 19)
Classes: typeBEvsBM ==>  (144, 19)


,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb
0,0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
1,1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
2,2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
3,3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1
4,4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1.0,57.4,17.7,4.4


In [26]:
## 2. Map typeBEvsBM
DF_2dfft6fold["typeBEvsBM"]= DF_2dfft6fold['Code'].map(output_dict)
print(DF_2dfft6fold.shape)
DF_2dfft6fold.head()

(144, 20)


/tmp/ipykernel_2783/284836721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dfft6fold["typeBEvsBM"]= DF_2dfft6fold['Code'].map(output_dict)


,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb,typeBEvsBM
0,0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
1,1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
2,2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
3,3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
4,4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1.0,57.4,17.7,4.4,M


In [30]:
DF_2dfft6fold.isnull().values.any()

False

In [31]:
print(set(DF_2dfft6fold["typeBEvsBM"]))

{'M', 'E', 'AEBart'}


In [33]:
DF_2dfft_EM = DF_2dfft6fold[DF_2dfft6fold["typeBEvsBM"]!='AEBart'].reset_index(drop=True)
print(DF_2dfft_EM.shape)
DF_2dfft_EM.head()

(133, 20)


,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,MCV,MCH,Hb,typeBEvsBM
0,0,0.0,0.0,0.0,237.0,237.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162433_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162433,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
1,1,1.0,1.0,1.0,234.0,234.0,HN35_D0_30HZ_20XINF_UWELL_25660131_161754_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_161754,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
2,2,4.0,4.0,4.0,239.0,239.0,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
3,3,8.0,8.0,8.0,240.0,240.0,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1.0,74.3,23.4,7.1,E
4,4,2.0,2.0,2.0,324.0,324.0,HN43_D0_30HZ_20XINF_UWELL_25660330_153259_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN43_D0_30HZ_20XINF_UWELL_25660330_153259,SSD_Backup,HN,HN43,No_Splenectomy,1,"[2, 5, 7]",1.0,57.4,17.7,4.4,M


In [34]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    lst_fold.sort()
    for j in range(len(lst_fold)):
        df_classes = DataSet[DataSet["fold"]==lst_fold[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        lst_folder = list(set(df_classes["fold"]))
        print(f"[INFO]: [{j+1}].  Fold {lst_fold[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold With Class:")
        lst_cls = list(set(df_classes['typeBEvsBM']))
        print(f"Number of Classes {len(lst_cls)} Class ==> {lst_cls}")
        for m in lst_cls:
            df_f =  df_classes[df_classes["typeBEvsBM"]==m]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]:  Classse {m} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*100)
    
    return

In [35]:
_(DF_2dfft_EM)

[INFO]: [1].  Fold 1 With Shape ==> 24 Folder, (24, 20)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 3 Folder, 3 images!! ==> 1 PerSon!!
[INFO]:  Classse E ==> With 21 Folder, 21 images!! ==> 5 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 24 Folder, (24, 20)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 9 Folder, 9 images!! ==> 2 PerSon!!
[INFO]:  Classse E ==> With 15 Folder, 15 images!! ==> 4 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 18 Folder, (18, 20)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 8 Folder, 8 images!! ==> 2 PerSon!!
[INFO]:  Classse E ==> With 10 Folder, 10 images!! ==> 3 PerSon!!
[INFO]: [4].  Fold 4 With Shape ==> 24 Folder, (24, 20)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 13 Folder, 13 images!! ==> 3 PerSon!!
[INFO]:  Classse E ==> With 11 Folder, 11 images!! ==> 3

> #### 3-fold

In [38]:
DF_3fold = pd.read_csv('/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-3Fold-split3class.csv')
print(DF_3fold.shape)
DF_3fold_ = DF_3fold[DF_3fold['subclass']!='Normal']
print("Classes: typeBEvsBM ==> ", DF_3fold_.shape)
DF_3fold_.head()
DF_3fold_.head()

(288, 15)
Classes: typeBEvsBM ==>  (192, 15)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary
0,0,0,0,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
1,1,1,1,269,269,HN38_D0_30HZ_20XINF_UWELL_25660223_145353_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145353,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
2,2,2,2,240,240,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1
3,3,3,3,265,265,HN38_D0_30HZ_20XINF_UWELL_25660223_144217_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144217,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1
4,4,4,4,266,266,HN38_D0_30HZ_20XINF_UWELL_25660223_144746_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144746,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1


In [39]:
## 3. Map typeBEvsBM
DF_3fold_["typeBEvsBM"]= DF_3fold_['Code'].map(output_dict)
print(DF_3fold_.shape)
DF_3fold_.head()

(192, 16)


/tmp/ipykernel_2783/3654070345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_3fold_["typeBEvsBM"]= DF_3fold_['Code'].map(output_dict)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,typeBEvsBM
0,0,0,0,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1,E
1,1,1,1,269,269,HN38_D0_30HZ_20XINF_UWELL_25660223_145353_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145353,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E
2,2,2,2,240,240,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1,E
3,3,3,3,265,265,HN38_D0_30HZ_20XINF_UWELL_25660223_144217_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144217,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E
4,4,4,4,266,266,HN38_D0_30HZ_20XINF_UWELL_25660223_144746_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144746,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E


In [40]:
print(set(DF_3fold_["typeBEvsBM"]))

{'M', 'E', 'AEBart'}


In [41]:
DF_3fold_EM = DF_3fold_[DF_3fold_["typeBEvsBM"]!='AEBart'].reset_index(drop=True)
print(DF_3fold_EM.shape)
DF_3fold_EM.head()

(180, 16)


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,fold,dt,classes_binary,typeBEvsBM
0,0,0,0,239,239,HN35_D0_30HZ_20XINF_UWELL_25660131_162844_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_162844,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1,E
1,1,1,1,269,269,HN38_D0_30HZ_20XINF_UWELL_25660223_145353_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_145353,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E
2,2,2,2,240,240,HN35_D0_30HZ_20XINF_UWELL_25660131_163058_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN35_D0_30HZ_20XINF_UWELL_25660131_163058,SSD_Backup,HN,HN35,No_Splenectomy,1,"[2, 5, 7]",1,E
3,3,3,3,265,265,HN38_D0_30HZ_20XINF_UWELL_25660223_144217_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144217,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E
4,4,4,4,266,266,HN38_D0_30HZ_20XINF_UWELL_25660223_144746_dt2_...,/media/tohn/HDD/rheology2023/Blood_Viscosity_2...,HN38_D0_30HZ_20XINF_UWELL_25660223_144746,SSD_Backup,HN,HN38,No_Splenectomy,1,"[2, 5, 7]",1,E


In [42]:
_(DF_3fold_EM)

[INFO]: [1].  Fold 1 With Shape ==> 64 Folder, (64, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 13 Folder, 13 images!! ==> 3 PerSon!!
[INFO]:  Classse E ==> With 51 Folder, 51 images!! ==> 9 PerSon!!
[INFO]: [2].  Fold 2 With Shape ==> 58 Folder, (58, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 27 Folder, 27 images!! ==> 5 PerSon!!
[INFO]:  Classse E ==> With 31 Folder, 31 images!! ==> 6 PerSon!!
[INFO]: [3].  Fold 3 With Shape ==> 58 Folder, (58, 16)
Check 6-Fold With Class:
Number of Classes 2 Class ==> ['M', 'E']
[INFO]:  Classse M ==> With 7 Folder, 7 images!! ==> 1 PerSon!!
[INFO]:  Classse E ==> With 51 Folder, 51 images!! ==> 9 PerSon!!


-------------------------------------------------------------------------------------------------------------------------------

### แบ่ง 3-fold cross validation 

E:M

3:1


[INFO]: Classes: E ==> 31

[INFO]: Classes: M ==> 11

In [50]:
df_dictCodeEM = df_dictCode[df_dictCode["typeBEvsBM"]!='AEBart'].reset_index(drop=True)
print(set(df_dictCode["typeBEvsBM"]))
print(df_dictCodeEM.shape)
df_dictCodeEM.head()

{'M', 'E', 'AEBart'}
(42, 6)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet
0,HN01,75.8,23.1,6.0,E,Splenectomy
1,HN02,73.5,22.4,7.2,E,Splenectomy
2,HN03,77.3,24.3,9.0,E,Splenectomy
3,HN04,79.8,25.6,7.6,E,Splenectomy
4,HN06,78.5,25.9,8.2,E,Splenectomy


In [55]:
31/3

10.333333333333334

In [56]:
11/3

3.6666666666666665

In [72]:
'''
fold1, fold2, fold3
E:M  , E:M  , E:M 
10   , 10   , 11
3    , 3    , 5
'''


'''
3:1  , 3:1 , 3.5:1.5  
'''

'\n3:1  , 3:1 , 3.5:1.5  \n'

In [53]:
EM_list = list(set(df_dictCodeEM["typeBEvsBM"]))
EM_list.sort()
EM_list

['E', 'M']

> #### 💾 Split -Fold

In [76]:
import random

EM_list = list(set(df_dictCodeEM["typeBEvsBM"]))
EM_list.sort()
nE = 10
nM = 3
DF_fold = pd.DataFrame(columns=list(df_dictCodeEM.columns))

for j in range(len(EM_list)):
    classes_n = EM_list[j]
    if classes_n == "E":
        n=10
    elif classes_n == "M":
         n=3
    df_classes = df_dictCodeEM[df_dictCodeEM["typeBEvsBM"]==classes_n]
    print(f"[INFO]: [{j+1}].  For Classes {classes_n} With Shape ==> {df_classes.shape}")
    for i in range(1, 4):
        df_classes_ran = df_classes.sample(n, replace=False)
        df_classes.drop(index=df_classes_ran.index, inplace=True)
        df_classes_ran["fold"] = i
        DF_fold = DF_fold.append(df_classes_ran)
        if i == 3:
            #print(df_classes.shape[0])
            break
    df_classes["fold"] = 3
    DF_fold = DF_fold.append(df_classes)

[INFO]: [1].  For Classes E With Shape ==> (31, 6)
[INFO]: [2].  For Classes M With Shape ==> (11, 6)


/tmp/ipykernel_2783/2305207773.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_2783/2305207773.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_fold = DF_fold.append(df_classes_ran)
/tmp/ipykernel_2783/2305207773.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_2783/2305207773.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [77]:
print(DF_fold.shape)
DF_fold

(42, 7)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet,fold
20,HN44,61.7,19.7,7.0,E,Splenectomy,1.0
3,HN04,79.8,25.6,7.6,E,Splenectomy,1.0
11,HN23,85.9,27.0,5.0,E,Splenectomy,1.0
1,HN02,73.5,22.4,7.2,E,Splenectomy,1.0
21,HN05,72.3,23.7,5.9,E,No_Splenectomy,1.0
26,HN14,78.8,25.2,5.7,E,No_Splenectomy,1.0
4,HN06,78.5,25.9,8.2,E,Splenectomy,1.0
37,HN35,74.3,23.4,7.1,E,No_Splenectomy,1.0
22,HN08,66.9,20.9,8.4,E,No_Splenectomy,1.0
34,HN31,67.7,21.3,6.0,E,No_Splenectomy,1.0


In [78]:
DF_fold = DF_fold.reset_index(drop=True)
print(DF_fold.shape)
DF_fold['fold'] = DF_fold['fold'].astype(int)
DF_fold.head()

(42, 7)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet,fold
0,HN44,61.7,19.7,7.0,E,Splenectomy,1
1,HN04,79.8,25.6,7.6,E,Splenectomy,1
2,HN23,85.9,27.0,5.0,E,Splenectomy,1
3,HN02,73.5,22.4,7.2,E,Splenectomy,1
4,HN05,72.3,23.7,5.9,E,No_Splenectomy,1


In [80]:
### Check fold
EM_list = list(set(DF_fold["typeBEvsBM"]))
EM_list.sort()
fold_list = list(set(DF_fold["fold"]))
fold_list.sort()
for j in range(len(fold_list)):
    fold_j = fold_list[j]
    df_fold_j = DF_fold[DF_fold["fold"]==fold_j]
    print(f"[INFO]: Fold-{fold_j}: {df_fold_j.shape[0]} Cases")
    for k in range(len(EM_list)):
        classes_k = EM_list[k]
        df_k = df_fold_j[df_fold_j["typeBEvsBM"]==classes_k]
        print(f"[INFO]: Classes: {classes_k} ==> {df_k.shape[0]}")
    print("="*125)

[INFO]: Fold-1: 13 Cases
[INFO]: Classes: E ==> 10
[INFO]: Classes: M ==> 3
[INFO]: Fold-2: 13 Cases
[INFO]: Classes: E ==> 10
[INFO]: Classes: M ==> 3
[INFO]: Fold-3: 16 Cases
[INFO]: Classes: E ==> 11
[INFO]: Classes: M ==> 5


In [81]:
13+13+16

42

In [83]:
## 1. Create Dict fold
lst_Code = DF_fold["Code"].tolist()
lst_fold = DF_fold["fold"].tolist()
dict_fold = zip(lst_Code, lst_fold)
output_fold = dict(dict_fold)
print(output_fold)

{'HN44': 1, 'HN04': 1, 'HN23': 1, 'HN02': 1, 'HN05': 1, 'HN14': 1, 'HN06': 1, 'HN35': 1, 'HN08': 1, 'HN31': 1, 'HN18': 2, 'HN09': 2, 'HN03': 2, 'HN12': 2, 'HN33': 2, 'HN21': 2, 'HN07': 2, 'HN27': 2, 'HN32': 2, 'HN26': 2, 'HN16': 3, 'HN13': 3, 'HN01': 3, 'HN42': 3, 'HN38': 3, 'HN29': 3, 'HN17': 3, 'HN39': 3, 'HN30': 3, 'HN20': 3, 'HN34': 3, 'HN41': 1, 'HN40': 1, 'HN22': 1, 'HN25': 2, 'HN28': 2, 'HN36': 2, 'HN43': 3, 'HN11': 3, 'HN10': 3, 'HN19': 3, 'HN24': 3}


In [95]:
## 2. Create Dict typeBEvsBM
lst_EM = DF_fold["typeBEvsBM"].tolist()
dict_EM = zip(lst_Code, lst_EM)
output_EM = dict(dict_EM)
print(output_EM)

{'HN44': 'E', 'HN04': 'E', 'HN23': 'E', 'HN02': 'E', 'HN05': 'E', 'HN14': 'E', 'HN06': 'E', 'HN35': 'E', 'HN08': 'E', 'HN31': 'E', 'HN18': 'E', 'HN09': 'E', 'HN03': 'E', 'HN12': 'E', 'HN33': 'E', 'HN21': 'E', 'HN07': 'E', 'HN27': 'E', 'HN32': 'E', 'HN26': 'E', 'HN16': 'E', 'HN13': 'E', 'HN01': 'E', 'HN42': 'E', 'HN38': 'E', 'HN29': 'E', 'HN17': 'E', 'HN39': 'E', 'HN30': 'E', 'HN20': 'E', 'HN34': 'E', 'HN41': 'M', 'HN40': 'M', 'HN22': 'M', 'HN25': 'M', 'HN28': 'M', 'HN36': 'M', 'HN43': 'M', 'HN11': 'M', 'HN10': 'M', 'HN19': 'M', 'HN24': 'M'}


In [99]:
DF_2dFFTOrigin = pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-2dFFTdataset-3channels-6Fold.csv")
del DF_2dFFTOrigin["fold"]
print(DF_2dFFTOrigin.shape)
DF_2dFFTOrigin_ = DF_2dFFTOrigin[DF_2dFFTOrigin['subclass']!='Normal']
print("Classes: Not Normal ==> ", DF_2dFFTOrigin_.shape)
DF_2dFFTOrigin_.head()

(538, 11)
Classes: Not Normal ==>  (337, 11)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1


In [100]:
## 3. Map typeBEvsBM
DF_2dFFTOrigin_["typeBEvsBM"]= DF_2dFFTOrigin_['Code'].map(output_EM)
print(set(DF_2dFFTOrigin_["typeBEvsBM"]))
DF_2dFFTOrigin_

{nan, 'M', 'E'}


/tmp/ipykernel_2783/629700225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dFFTOrigin_["typeBEvsBM"]= DF_2dFFTOrigin_['Code'].map(output_EM)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
...,...,...,...,...,...,...,...,...,...,...,...,...
332,332,332,HN44_D0_30HZ_20XINF_UWELL_25660330_155808_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_155808,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,E
333,333,333,HN44_D0_30HZ_20XINF_UWELL_25660330_160007_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160007,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,E
334,334,334,HN44_D0_30HZ_20XINF_UWELL_25660330_160337_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160337,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,E
335,335,335,HN44_D0_30HZ_20XINF_UWELL_25660330_160604_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160604,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,E


In [104]:
DF_2dFFTEM = DF_2dFFTOrigin_[DF_2dFFTOrigin_['typeBEvsBM'].notna()]
print(set(DF_2dFFTEM["typeBEvsBM"]))
print(DF_2dFFTEM.shape)
DF_2dFFTEM.head()

{'M', 'E'}
(317, 12)


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,typeBEvsBM
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,E


In [105]:
DF_2dFFTEM.isnull().values.any()

False

In [108]:
list_code_2dfft = list(set(DF_2dFFTEM["Code"]))
list_code_2dfft.sort()
print("Number of Cases: ", len(list_code_2dfft))

Number of Cases:  33


In [109]:
list_code_2dfft

['HN01',
 'HN02',
 'HN12',
 'HN13',
 'HN14',
 'HN16',
 'HN17',
 'HN18',
 'HN19',
 'HN20',
 'HN21',
 'HN22',
 'HN23',
 'HN24',
 'HN25',
 'HN26',
 'HN27',
 'HN28',
 'HN29',
 'HN30',
 'HN31',
 'HN32',
 'HN33',
 'HN34',
 'HN35',
 'HN36',
 'HN38',
 'HN39',
 'HN40',
 'HN41',
 'HN42',
 'HN43',
 'HN44']

In [110]:
print(df_dictCodeEM.shape)
df_dictCodeEM.head()

(42, 6)


,Code,MCV,MCH,Hb,typeBEvsBM,sheet
0,HN01,75.8,23.1,6.0,E,Splenectomy
1,HN02,73.5,22.4,7.2,E,Splenectomy
2,HN03,77.3,24.3,9.0,E,Splenectomy
3,HN04,79.8,25.6,7.6,E,Splenectomy
4,HN06,78.5,25.9,8.2,E,Splenectomy


In [116]:
DF_B_res_ML= pd.read_csv("/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_split3class.csv")
del DF_B_res_ML["fold"]
print(DF_B_res_ML.shape)
DF_B_res_ML.head()

(51, 8)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4
3,3,NBL14,NBL,Normal,0.0,86.8,28.1,13.2
4,4,NBL23,NBL,Normal,0.0,66.5,20.3,13.0


In [117]:
DFML=DF_B_res_ML[DF_B_res_ML["subclass"]!="Normal"].reset_index(drop=True)
print(DFML.shape)
DFML.head()

(35, 8)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4
3,6,HN17,HN,Splenectomy,1.0,75.0,25.0,7.1
4,7,HN12,HN,Splenectomy,1.0,75.4,24.1,9.8


In [118]:
DFML["typeBEvsBM"]= DFML['Code'].map(output_EM)
print(DFML.shape)
DFML.head()

(35, 9)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1,E
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0,E
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4,M
3,6,HN17,HN,Splenectomy,1.0,75.0,25.0,7.1,E
4,7,HN12,HN,Splenectomy,1.0,75.4,24.1,9.8,E


In [121]:
set(DFML["typeBEvsBM"])

{'E', 'M', nan}

In [119]:
set(DFML["Code"]) - set(list_code_2dfft)

{'HN15', 'HN37'}

In [120]:
set(list_code_2dfft) - set(DFML["Code"])

set()

In [122]:
DFML_EM = DFML[DFML['typeBEvsBM'].notna()]
print(set(DFML_EM["typeBEvsBM"]))
print(DFML_EM.shape)
DFML_EM.head()

{'M', 'E'}
(33, 9)


,Unnamed: 0,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM
0,0,HN35,HN,No_Splenectomy,1.0,74.3,23.4,7.1,E
1,1,HN38,HN,No_Splenectomy,1.0,72.9,22.6,8.0,E
2,2,HN43,HN,No_Splenectomy,1.0,57.4,17.7,4.4,M
3,6,HN17,HN,Splenectomy,1.0,75.0,25.0,7.1,E
4,7,HN12,HN,Splenectomy,1.0,75.4,24.1,9.8,E


In [123]:
EM_list = list(set(DFML_EM["typeBEvsBM"]))
EM_list.sort()
for j in range(len(EM_list)):
    classes_j = EM_list[j]
    df_j = DFML_EM[DFML_EM["typeBEvsBM"]==classes_j]
    print(f"[INFO]: Classes: {classes_j} ==> {df_j.shape[0]}")

[INFO]: Classes: E ==> 24
[INFO]: Classes: M ==> 9


In [126]:
"""
fold-1, fold-2, fold-3
E:M, E:M, E:M,
8:3, 8:3, 8:3

"""

'\nfold-1, fold-2, fold-3\nE:M, E:M, E:M,\n8:3, 8:3, 8:3\n\n'

In [127]:
8+8+8

24

In [128]:
3+3+3

9

In [88]:
## 3. Map typeBEvsBM
DF_2dFFTOrigin_["fold"]= DF_2dFFTOrigin_['Code'].map(output_fold)
print(set(DF_2dFFTOrigin_["fold"]))

{nan, 1.0, 2.0, 3.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}


/tmp/ipykernel_2783/1414372353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dFFTOrigin_["fold"]= DF_2dFFTOrigin_['Code'].map(output_fold)


In [89]:
DF_2dFFTOrigin_.head()

,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,fold
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3.0
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3.0
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3.0
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3.0
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3.0


In [90]:
DF_2dFFTOrigin_.isnull().values.any()

True

In [94]:
DF_2dFFT3fold = DF_2dFFTOrigin_[DF_2dFFTOrigin_['fold'].notna()]
DF_2dFFT3fold["fold"] = DF_2dFFT3fold["fold"].astype('int')
print(DF_2dFFT3fold.shape)
print("Fold: ", set(DF_2dFFT3fold["fold"]))
print("Number of Cases: ", len(list(set(DF_2dFFT3fold["Code"]))))
DF_2dFFT3fold

(317, 12)
Fold:  {1, 2, 3}
Number of Cases:  33


/tmp/ipykernel_2783/2822078133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_2dFFT3fold["fold"] = DF_2dFFT3fold["fold"].astype('int')


,Unnamed: 0.1,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,Code,subclass,dt,classes_binary,fold
0,0,0,HN01_D0_30HZ_20XINF_UWELL_20221027_154439_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154439,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3
1,1,1,HN01_D0_30HZ_20XINF_UWELL_20221027_154831_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_154831,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3
2,2,2,HN01_D0_30HZ_20XINF_UWELL_20221027_155220_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155220,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3
3,3,3,HN01_D0_30HZ_20XINF_UWELL_20221027_155642_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_155642,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3
4,4,4,HN01_D0_30HZ_20XINF_UWELL_20221027_160124_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN01_D0_30HZ_20XINF_UWELL_20221027_160124,SSD_Backup,HN,HN01,Splenectomy,"[2, 5, 7]",1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
332,332,332,HN44_D0_30HZ_20XINF_UWELL_25660330_155808_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_155808,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,1
333,333,333,HN44_D0_30HZ_20XINF_UWELL_25660330_160007_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160007,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,1
334,334,334,HN44_D0_30HZ_20XINF_UWELL_25660330_160337_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160337,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,1
335,335,335,HN44_D0_30HZ_20XINF_UWELL_25660330_160604_dt2_...,/media/HDD/rheology2023/Blood_Viscosity_2dFFT/...,HN44_D0_30HZ_20XINF_UWELL_25660330_160604,SSD_Backup,HN,HN44,Splenectomy,"[2, 5, 7]",1,1
